In [1]:
import pandas as pd
import backtrader as bt
from datetime import datetime
from loguru import logger

In [2]:
class CrossOver(bt.Strategy):
    # параметры для скользящих средних
    params = dict(
        fast_ma=10,
        slow_ma=20,
    )

    def __init__(self):
        '''рассчёт скользящих средних и индикатора их пересечения'''
        fast_ma = bt.ind.SMA(period=self.p.fast_ma)
        slow_ma = bt.ind.SMA(period=self.p.slow_ma)
        self.crossover = bt.ind.CrossOver(fast_ma, slow_ma)
        self.order = None # для хранения состояния заявки

    def notify_order(self, order):
        '''вывод сделок в консоль'''
        if order.status in [order.Submitted, order.Accepted]:
            return
        
        if order.status in [order.Completed]:
            if order.isbuy():
                logger.info(f'BUY@price {order.executed.price}')
            elif order.issell():
                logger.info(f'SELL@price {order.executed.price}')
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            logger.warning('Order Canceled/Margin/Rejected')

        self.order = None

    def next(self):
        '''cам алгоритм'''
        if self.order:
            # если прошлая заявка не исполнена, то ничего не делать
            return

        if not self.position:
            if self.crossover > 0:
                self.order = self.buy()
        elif self.crossover < 0:
                self.order = self.close()

In [3]:
cerebro = bt.Cerebro()
commission = 0.05
cerebro.broker.setcommission(commission=commission/100, leverage=1)
cerebro.broker.setcash(10000.) # стартовый кэш
cerebro.addsizer(bt.sizers.PercentSizer, percents=95) # размер позиции

df = pd.read_csv(
    '..//data//1d//AAPL.csv', # файл с котировками
    index_col=0,
    parse_dates=True
)

data = bt.feeds.PandasData(
    dataname=df,
    openinterest=-1,
    fromdate=datetime(2005, 1, 1), # стартовая дата
    # todate=datetime(2005, 2, 1),
    timeframe=bt.TimeFrame.Days,
    compression=1
)

cerebro.adddata(data) # добавляем котировки
cerebro.addstrategy(CrossOver) # добавляем стратегию
cerebro.run() # запускаем рассчёты

2021-10-12 11:52:25.601 | INFO     | __main__:notify_order:22 - BUY@price 1.5199999809265134
2021-10-12 11:52:25.609 | INFO     | __main__:notify_order:24 - SELL@price 1.5175000429153442
2021-10-12 11:52:25.621 | INFO     | __main__:notify_order:22 - BUY@price 1.264999985694885
2021-10-12 11:52:25.626 | INFO     | __main__:notify_order:24 - SELL@price 1.2342859506607056
2021-10-12 11:52:25.638 | INFO     | __main__:notify_order:22 - BUY@price 1.4107140302658079
2021-10-12 11:52:25.649 | INFO     | __main__:notify_order:24 - SELL@price 1.2828569412231443
2021-10-12 11:52:25.661 | INFO     | __main__:notify_order:22 - BUY@price 1.3296430110931394
2021-10-12 11:52:25.672 | INFO     | __main__:notify_order:24 - SELL@price 1.3703570365905762
2021-10-12 11:52:25.679 | INFO     | __main__:notify_order:22 - BUY@price 1.4632140398025513
2021-10-12 11:52:25.700 | INFO     | __main__:notify_order:24 - SELL@price 1.5521429777145386
2021-10-12 11:52:25.704 | INFO     | __main__:notify_order:22 - BU

In [4]:
cerebro.plot(iplot=False) # рисуем графики

[[<Figure size 1102x511 with 5 Axes>]]